# Inaugural project

## Purpose
In this project we'll seek to solve the Black-Scholes model of a simulated underlying asset $S$

## Theory



In [381]:
import numpy as np
import matplotlib.pyplot as plt
from math import exp as exp
import plotly.express as px
import pandas as pd
from scipy.stats import norm

In [382]:
## Define a function that simulate the stock price
def stock_price_sim(S0=100, mu=0.01, sigma=0.08, T=500):
    """
    Simulation of stock price according to Black-Scholes model.
    
    The function takes following arguments:
    S0: Initial stock value
    mu: Average stock price return
    sigma: Stock volatility
    T: Length of stock price history
    """
    
    # Set seed
    np.random.seed(1)
    
    # Calculate dt≈Delta t=1/T
    dt = 1/T
    
    # Create list of standard normal distributed values
    epsilon = [np.random.normal(0,1) for x in range(T)]
    
    # Daily log returns
    r = [(mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * epsilon[i] for i in range(T)]
    
    # Stock price evolution
    S = []
    for t in range(T):
        S_t = S0 * exp(sum(r[0:t+1]))
        S.append(S_t)
        
    # Create dataframe from list
    S_df = pd.DataFrame(S, columns = ['Stock Price'])
        
    # Return stock price data
    return S_df   

In [429]:
## Plot stock
stock_price = stock_price_sim()
fig = px.line(stock_price, title='Evolution of the underlying stock price',
             labels=dict(value="Stock Price", index="$t$"))
fig.show()

In [393]:
## Define a option that will calculate price of eu call option on non dividend paying stock
def call_price(K, r, T, S_t):
    """
    This functions calculates the price on a european call option on a 
    non dividend paying stock and takes the arguments:
    
    K: Strike price
    r: Risk free rate
    T: Maturity date given as a fraction of a year
    S_t: A df containing stock price history
    """
    
    # Define the standard normal function
    N = norm.cdf
    
    # Define sigma
    sigma = stock_price.diff().std()[0]
    
    # Define d1 and d2
    d1 = (np.log(S_t.iloc[-1]/K) + (r+0.5*sigma**2) * (T)) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    
    # Calculate option price
    c_t = S_t.iloc[-1]*N(d1)-K*exp(-r*(T))*N(d2)
    
    # Return price
    return c_t[0]

In [412]:
## Call price for different values of K
call_price_list = []
for i in range(300):
    c = call_price(i, 0.1, 0.25, stock_price)
    call_price_list.append(c)

In [428]:
## Plot stock
call_price_df = pd.DataFrame(call_price_list, columns = ['Call Price'])
fig = px.line(call_price_df, title='Call price as a function of strike price (K)', 
              labels=dict(value="Call Price", index="$K$"))
fig.show()